# Model Evaluation Notebook
# Notebook untuk evaluasi komprehensif model machine learning

## 1. Import Library yang Dibutuhkan

In [ ]:
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics import (accuracy_score, f1_score, confusion_matrix, 
                            classification_report, precision_score, recall_score,
                            roc_auc_score, roc_curve, precision_recall_curve)
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import json
import warnings
warnings.filterwarnings('ignore')

## 2. Fungsi Load Model

In [ ]:
def load_model(model_path):
    """
    Load model dari file .pkl atau .joblib
    
    Args:
        model_path: Path ke file model
    
    Returns:
        model: Model yang telah di-load atau None jika gagal
    """
    try:
        # Coba load dengan pickle
        with open(model_path, 'rb') as f:
            model = pickle.load(f)
        print(f"✅ Model berhasil dimuat dari {model_path}")
        return model
    except FileNotFoundError:
        print(f"❌ File {model_path} tidak ditemukan!")
        return None
    except Exception as e:
        # Coba dengan joblib jika pickle gagal
        try:
            import joblib
            model = joblib.load(model_path)
            print(f"✅ Model berhasil dimuat dari {model_path} (menggunakan joblib)")
            return model
        except Exception as e2:
            print(f"❌ Error saat memuat model: {e}")
            return None

## 3. Fungsi Evaluasi Model Umum

In [ ]:
def evaluate_model(model, X_test, y_test, target_names=None):
    """
    Evaluasi model dengan metrik standar
    
    Args:
        model: Model yang akan dievaluasi
        X_test: Features test set
        y_test: Labels test set
        target_names: Nama kelas untuk classification report
    
    Returns:
        dict: Dictionary berisi semua metrik evaluasi
    """
    print("🔍 Melakukan prediksi...")
    y_pred = model.predict(X_test)
    
    # Hitung metrik dasar
    accuracy = accuracy_score(y_test, y_pred)
    
    # F1 score - sesuaikan berdasarkan jenis klasifikasi
    unique_classes = len(np.unique(y_test))
    if unique_classes == 2:
        f1 = f1_score(y_test, y_pred, average='binary')
        precision = precision_score(y_test, y_pred, average='binary')
        recall = recall_score(y_test, y_pred, average='binary')
    else:
        f1 = f1_score(y_test, y_pred, average='weighted')
        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
    
    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    
    # Print hasil
    print("=" * 60)
    print("📊 HASIL EVALUASI MODEL")
    print("=" * 60)
    print(f"🎯 Akurasi: {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"🎯 Precision: {precision:.4f}")
    print(f"🎯 Recall: {recall:.4f}")
    print(f"🎯 F1 Score: {f1:.4f}")
    print(f"📋 Jumlah prediksi: {len(y_pred)}")
    print(f"📋 Jumlah kelas: {unique_classes}")
    
    print("\n📈 Confusion Matrix:")
    print(cm)
    
    # Classification Report
    print("\n📊 Classification Report:")
    print(classification_report(y_test, y_pred, target_names=target_names))
    
    # Return results
    results = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'confusion_matrix': cm.tolist(),
        'y_test': y_test.tolist() if hasattr(y_test, 'tolist') else list(y_test),
        'y_pred': y_pred.tolist() if hasattr(y_pred, 'tolist') else list(y_pred),
        'classification_report': classification_report(y_test, y_pred, target_names=target_names, output_dict=True)
    }
    
    return results

## 4. Visualisasi Confusion Matrix

In [ ]:

def plot_confusion_matrix(cm, class_names=None, title='Confusion Matrix', figsize=(8, 6)):
    """
    Visualisasi confusion matrix dengan heatmap
    
    Args:
        cm: Confusion matrix
        class_names: List nama kelas
        title: Judul plot
        figsize: Ukuran figure
    """
    plt.figure(figsize=figsize)
    
    if class_names is None:
        class_names = [f'Class {i}' for i in range(len(cm))]
    
    # Heatmap
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names,
                square=True, linewidths=0.5, cbar_kws={"shrink": .8})
    
    plt.title(title, fontsize=14, fontweight='bold')
    plt.xlabel('Predicted Label', fontsize=12)
    plt.ylabel('True Label', fontsize=12)
    plt.tight_layout()
    plt.show()
    
    # Print accuracy per class
    print("\n📊 Accuracy per Class:")
    total_per_class = cm.sum(axis=1)
    correct_per_class = np.diag(cm)
    
    for i, class_name in enumerate(class_names):
        if total_per_class[i] > 0:
            class_accuracy = correct_per_class[i] / total_per_class[i]
            print(f"  {class_name}: {class_accuracy:.4f} ({class_accuracy*100:.2f}%)")

## 5. Fungsi Save Hasil Evaluasi

In [ ]:
def save_evaluation_results(results, filename='evaluation_results.json'):
    """
    Simpan hasil evaluasi ke file JSON
    
    Args:
        results: Dictionary hasil evaluasi
        filename: Nama file output
    """
    try:
        with open(filename, 'w') as f:
            json.dump(results, f, indent=2)
        print(f"✅ Hasil evaluasi disimpan ke '{filename}'")
    except Exception as e:
        print(f"❌ Error saat menyimpan hasil: {e}")

## 6. Contoh Penggunaan dengan Data Dummy

In [ ]:
def create_demo_data():
    """
    Buat data dummy untuk demonstrasi
    """
    from sklearn.datasets import make_classification
    from sklearn.ensemble import RandomForestClassifier
    
    print("🔧 Membuat data dummy untuk demonstrasi...")
    
    # Generate data
    X, y = make_classification(n_samples=1000, n_features=20, n_classes=3, 
                                n_informative=15, n_clusters_per_class=1, random_state=42)
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                                        stratify=y, random_state=42)
    
    # Train model
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    print("✅ Data dummy dan model berhasil dibuat")
    
    return model, X_test, y_test

## 10. Main Execution - Contoh Lengkap

In [ ]:
def main_evaluation_example():
    """
    Contoh lengkap evaluasi model
    """
    print("=" * 60)
    print("🚀 CONTOH EVALUASI MODEL LENGKAP")
    print("=" * 60)
    
    # 1. Load Model (ganti dengan path model Anda)
    model_path = 'your_model.pkl'  # Ganti dengan path model Anda
    model = load_model(model_path)
    
    if model is None:
        print("📝 Model tidak ditemukan, menggunakan data dummy...")
        model, X_test, y_test = create_demo_data()
        class_names = ['Class A', 'Class B', 'Class C']
    else:
        # 2. Load Data Test Anda
        print("📝 Silakan load data test Anda:")
        print("   X_test = pd.read_csv('your_test_features.csv')")
        print("   y_test = pd.read_csv('your_test_labels.csv')")
        print("   Untuk demo, akan menggunakan data dummy...")
        
        _, X_test, y_test = create_demo_data()
        class_names = ['Class A', 'Class B', 'Class C']
    
    # 3. Evaluasi Komprehensif
    results = comprehensive_evaluation(model, X_test, y_test, class_names)
    
    # 4. Plot Confusion Matrix
    cm = np.array(results['confusion_matrix'])
    plot_confusion_matrix(cm, class_names, 'Final Confusion Matrix')
    
    print("\n✅ Evaluasi selesai!")
    return results

# Jalankan contoh evaluasi
print("=" * 60)
print("📋 PANDUAN PENGGUNAAN NOTEBOOK EVALUASI MODEL")
print("=" * 60)
print("""
1. Ganti 'model_path' dengan path ke model Anda
2. Load data test Anda (X_test, y_test)
3. Tentukan nama kelas (class_names)
4. Jalankan fungsi comprehensive_evaluation()

Untuk menjalankan demo:
results = main_evaluation_example()
""")